# Library

In [1]:
# install requirements (uncomment if you need it)
#!pip install request
#!pip install beautifulsoup4
#!pip install pandas

In [2]:
# importing library
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# URL & Base Models

In [3]:
# base soup models
def model(base_url, flex_url):
    response = requests.get(base_url+flex_url)
    
    # table soup model
    soup = BeautifulSoup(response.text, 'html')
    return soup

In [4]:
# function to extract the base element
def base_elements(element, base_url, flex_url):
    soup = model(base_url, flex_url)
    base_element = soup.find_all(f'{element}')
    return base_element

In [5]:
# functon to get the needed sub_elements
def sub_elements(base_element, sub_element):
    all_data = []
    for table in base_element:
        data = table.find_all(f'{sub_element}')
        all_data.append(data)
    return data

# Creating Headers of Dataframe

In [6]:
# make table as base elements
base_url = 'https://www.planecrashinfo.com/'
flex_url = 'lastwords.htm'
tables = base_elements('table', base_url, flex_url)

In [7]:
# get the headers
tb_headers = sub_elements(tables, 'tr')

# get the headers only
tb_headers = tb_headers[0]

In [8]:
# get the value from th elements
tb_headers_titles = [title.text.strip() for title in tb_headers]
tb_headers_titles = [item for item in tb_headers_titles if item]
tb_headers_titles

['TRANSCRIPT', 'AIRLINE', 'FLIGHT']

In [9]:
# append new value into the headers
tb_headers_titles.insert(0, 'BY')
tb_headers_titles.append('CONVERSATION')
tb_headers_titles

['BY', 'TRANSCRIPT', 'AIRLINE', 'FLIGHT', 'CONVERSATION']

In [10]:
# importing titles into dataframe
df = pd.DataFrame(columns = tb_headers_titles)
df

,BY,TRANSCRIPT,AIRLINE,FLIGHT,CONVERSATION


# Creating Column Data of Dataframe

In [11]:
# extract the column data
column_data = sub_elements(tables, 'tr')

In [12]:
for row in column_data[1:]:
    row_data = row.find_all('td')
    row_data_list = [data.text.strip() for data in row_data]
    
    # insert into df
    length = len(df)
    df.loc[length] = row_data_list

In [13]:
df.tail(5)

,BY,TRANSCRIPT,AIRLINE,FLIGHT,CONVERSATION
92,ATC,15 Jan 2009,US Airways,1549,We're gonna be in the Hudson.
93,,12 Feb 2009,Continental Express,3407,We're down.
94,CVR,10 Apr 2010,Polish Air Force,1549,F**kkkkkk.
95,CVR,07 Sept 2011,Yak service flight,9633,That's it we are f**ked up.
96,CVR,12 Dec 2019,Bek Air,2100,F**k your...ah.


In [14]:
df.insert(0, 'ID', range(1, len(df) + 1))

## Get The Link of Second Page

In [15]:
base_url = 'https://www.planecrashinfo.com/'
href = []
for row in column_data[1:]:
    second_column = row.find_all('td')[1]
    link = second_column.find_all('a')
    link = [link.get('href') for link in link]
    href.append(link)

In [16]:
# inserting the list into df
df['HREF'] = href

In [17]:
# clean the list before inserting into df
def clean_href_column(href):
    if isinstance(href, list) and len(href) == 1:
        return href[0]  
    elif isinstance(href, list) and len(href) == 0:
        return np.nan 
    return href

# Apply the function to the 'HREF' column
df['HREF'] = df['HREF'].apply(clean_href_column)

In [18]:
df = df.fillna(0)
df.tail(5)

,ID,BY,TRANSCRIPT,AIRLINE,FLIGHT,CONVERSATION,HREF
92,93,ATC,15 Jan 2009,US Airways,1549,We're gonna be in the Hudson.,cvr090115.htm
93,94,,12 Feb 2009,Continental Express,3407,We're down.,cvr090212.htm
94,95,CVR,10 Apr 2010,Polish Air Force,1549,F**kkkkkk.,https://www.youtube.com/watch?v=0JNEZlLceVk
95,96,CVR,07 Sept 2011,Yak service flight,9633,That's it we are f**ked up.,0
96,97,CVR,12 Dec 2019,Bek Air,2100,F**k your...ah.,0


In [19]:
mask = df['HREF'].str.contains('https://').fillna(False)
df.loc[mask, 'HREF'] = 0
df.tail(5)

,ID,BY,TRANSCRIPT,AIRLINE,FLIGHT,CONVERSATION,HREF
92,93,ATC,15 Jan 2009,US Airways,1549,We're gonna be in the Hudson.,cvr090115.htm
93,94,,12 Feb 2009,Continental Express,3407,We're down.,cvr090212.htm
94,95,CVR,10 Apr 2010,Polish Air Force,1549,F**kkkkkk.,0
95,96,CVR,07 Sept 2011,Yak service flight,9633,That's it we are f**ked up.,0
96,97,CVR,12 Dec 2019,Bek Air,2100,F**k your...ah.,0


In [20]:
link_list = []
for index, row in df.iterrows():
    if row['HREF'] != 0:
        link_list.append(row['HREF'])
    else:
        link_list.append('')

In [21]:
# func to extract data from second page
def extracting_data(flex_url):
    # make table as base elements
    base_url = 'https://www.planecrashinfo.com/'
    html = base_elements('html', base_url, flex_url)
    
    # extract the column data
    for element in html:
        if element.find_all('tr'):
            column_data = sub_elements(html, 'tr')
        elif element.find_all('ol'):
            column_data = sub_elements(html, 'ol')
        elif element.find_all('font'):
            column_data = sub_elements(html, 'font')
        else:
            column_data = sub_elements(html, 'body')
            
    # extract conversation
    conversation = []
    for row in column_data:
        if row.find_all('td'):
            row_data = row.find_all('td')
            row_data_list = [data.text.strip() for data in row_data]
        elif row.find_all('p'):
            row_data = row.find_all('p')
            row_data_list = [data.text.strip() for data in row_data]
        else:
            row_data_list = [data.text.strip() for data in row]
            
        conversation.append(row_data_list)
    return conversation

In [22]:
conversation = []
for i in range(len(link_list)):
    if link_list[i] == '':
        conv = df[df['ID'] == i+1]
        text = conv['CONVERSATION']
        df['CONVERSATION'][i] = text.to_string(index=False)
    else:
        data = extracting_data(link_list[i])
        df['CONVERSATION'][i] = data

C:\Users\koman\AppData\Local\Temp\ipykernel_26488\3554560146.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CONVERSATION'][i] = data
C:\Users\koman\AppData\Local\Temp\ipykernel_26488\3554560146.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CONVERSATION'][i] = text.to_string(index=False)


In [24]:
df.tail(5)

,ID,BY,TRANSCRIPT,AIRLINE,FLIGHT,CONVERSATION,HREF
92,93,ATC,15 Jan 2009,US Airways,1549,"[[January 15, 2009\n New York, New York\n US...",cvr090115.htm
93,94,,12 Feb 2009,Continental Express,3407,"[[21:18:22, , ], [START OF TRANSCRIPT, , ], [1...",cvr090212.htm
94,95,CVR,10 Apr 2010,Polish Air Force,1549,F**kkkkkk.,0
95,96,CVR,07 Sept 2011,Yak service flight,9633,That's it we are f**ked up.,0
96,97,CVR,12 Dec 2019,Bek Air,2100,F**k your...ah.,0


In [23]:
# uncoment if you want to get the csv
# df.to_csv("last_conversation.csv", index=False)